# Лабораторная работа №1

In [1]:
import pandas as pd
import pymorphy2

from tqdm import tqdm
tqdm.pandas()

import nltk
# nltk.download('punkt')

import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
import eli5

### Предобработка датасета

In [2]:
"""
Считываем наши данные и оформляем в DataFrame.
"""
df = pd.read_csv('women-clothing-accessories.3-class.balanced.csv', encoding="utf8", sep='\t')
df

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [3]:
df['sentiment'].value_counts()

sentiment
negative    30000
neautral    30000
positive    30000
Name: count, dtype: int64

In [4]:
"""
Мы оставляем тоьлко положительные и отрицательные отзывы.
"""
df = df[df['sentiment'] != 'neautral']
print(df.iloc[0]['sentiment'])
print(df.iloc[0]['review'])


negative
качество плохое пошив ужасный (горловина наперекос) Фото не соответствует Ткань ужасная рисунок блеклый маленький рукав не такой УЖАС!!!!! не стоит за такие деньги г.......


In [5]:
"""
Делаем дальнейшую предобработку отзывов.
Убираем все занки препинания.
"""
df['review_processed'] = df['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['review_processed'][0]

C:\Users\Sofia\AppData\Local\Temp\ipykernel_4940\3484675122.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_processed'] = df['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


'качество плохое пошив ужасный горловина наперекос Фото не соответствует Ткань ужасная рисунок блеклый маленький рукав не такой УЖАС не стоит за такие деньги г'

In [6]:
"""
Токенизируем наши отзывы.
"""
df['review_processed'] = df['review_processed'].progress_apply(lambda x: nltk.word_tokenize(x))
df['review_processed'].iloc[0]

100%|██████████| 60000/60000 [00:04<00:00, 12300.47it/s]
C:\Users\Sofia\AppData\Local\Temp\ipykernel_4940\3496717151.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_processed'] = df['review_processed'].progress_apply(lambda x: nltk.word_tokenize(x))


['качество',
 'плохое',
 'пошив',
 'ужасный',
 'горловина',
 'наперекос',
 'Фото',
 'не',
 'соответствует',
 'Ткань',
 'ужасная',
 'рисунок',
 'блеклый',
 'маленький',
 'рукав',
 'не',
 'такой',
 'УЖАС',
 'не',
 'стоит',
 'за',
 'такие',
 'деньги',
 'г']

In [7]:
"""
Приводим все слова к нормальной форме. 
"""
morph = pymorphy2.MorphAnalyzer()
df['review_lemmatized'] = df['review_processed'].progress_apply(lambda x: [morph.parse(w)[0].normal_form for w in x])

100%|██████████| 60000/60000 [02:04<00:00, 480.71it/s]
C:\Users\Sofia\AppData\Local\Temp\ipykernel_4940\4180616214.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_lemmatized'] = df['review_processed'].progress_apply(lambda x: [morph.parse(w)[0].normal_form for w in x])


In [8]:
"""
TfIdf
Определяем веса(важность) для каждого слова или словосочетания.
"""
vectorizer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True) # До этого был параметр ngram_range=(1, 1); lowercase - переводит все токены в нижний регистр
X = vectorizer.fit_transform(df['review_lemmatized'].apply(lambda x: ' '.join(x)))
X.shape


(60000, 396100)

### Обучение модели (логистическая регрессия)

In [9]:
"""
Разделяем на обучающую и тестовую выборку.
"""
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)
"""
Обучаем нашу модель.
"""
logit = LogisticRegression()
logit.fit(X_train, y_train)


LogisticRegression()

### Оценка модели

In [10]:
y_pred = logit.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)

0.9777779791573166

In [11]:
"""
Визуализация весов.
"""
eli5.show_weights(estimator=logit, 
                  feature_names= list(vectorizer.get_feature_names()),
                 top=(50, 50))

c:\Users\Sofia\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+10.460,отличный
+10.075,хороший
+7.917,супер
+7.339,спасибо
+7.282,немного
+6.904,хорошо
+6.754,отлично
+6.230,приятный
+6.177,классный
+6.061,довольный


### Пробуем рассмотреть не только слова, но и словосочетания с помощью Phrases

In [12]:
from gensim.models import Phrases
bigram = Phrases(df['review_lemmatized'].values, min_count=2, threshold=1)
bigram[df['review_lemmatized'].iloc[0]]

['качество_плохой',
 'пошив_ужасный',
 'горловина',
 'наперекос',
 'фото',
 'не_соответствовать',
 'ткань_ужасный',
 'рисунок',
 'блёклый',
 'маленький',
 'рукав',
 'не',
 'такой_ужас',
 'не_стоить',
 'за_такой',
 'деньга',
 'г']

In [13]:
"""
Применяем Phrases ко всем отзывам
"""
df['review_lemmatized_coll'] = df['review_lemmatized'].progress_apply(lambda x: bigram[x])

100%|██████████| 60000/60000 [00:01<00:00, 48813.19it/s]
C:\Users\Sofia\AppData\Local\Temp\ipykernel_4940\1856973362.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_lemmatized_coll'] = df['review_lemmatized'].progress_apply(lambda x: bigram[x])


### Пробуем обучить на новых данных

In [14]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1), lowercase=True) 
X = vectorizer.fit_transform(df['review_lemmatized_coll'].apply(lambda x: ' '.join(x)))
X.shape


(60000, 73017)

In [15]:
"""
Разделяем на обучающую и тестовую выборку.
"""
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)
"""
Обучаем нашу модель.
"""
logit = LogisticRegression()
logit.fit(X_train, y_train)

LogisticRegression()

### Оценка модели

In [16]:
y_pred = logit.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)

0.9726146496239656

In [17]:
"""
Визуализация весов.
"""
eli5.show_weights(estimator=logit, 
                  feature_names= list(vectorizer.get_feature_names()),
                 top=(50, 50))

c:\Users\Sofia\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+5.688,спасибо_продавец
+5.527,отличный
+5.506,супер
+5.039,отлично
+5.037,рекомендовать
+5.011,немного
+4.761,спасибо
+4.737,очень_довольный
+4.686,очень_хороший
+4.662,качество_отличный


### С помомщью GridSearchCV подбираем наилучшие параметры для нашей логистической решрессиии

In [18]:
parameters = {'C': [0.5, 1, 10, 20, 100],
              'max_iter': [50, 100, 300, 500]}
clf = GridSearchCV(logit, parameters)
clf.fit(X_train, y_train)
clf.best_params_

c:\Users\Sofia\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Sofia\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

{'C': 10, 'max_iter': 50}

### Переобучим нашу модель на наилучших параметрах из предыдущего пункта и посмотрим на результат 

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)
"""
Обучаем нашу модель.
"""
logit = LogisticRegression(C=10, max_iter=50)
logit.fit(X_train, y_train)

y_pred = logit.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_pred)

c:\Users\Sofia\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9759656312125862

### Таким образом, точность модели улучшилась на 0.3%